# Task 5: Model Interpretability


In [ ]:
# Install necessary libraries
!pip install shap lime transformers datasets

# Import required libraries
import shap
from lime.lime_text import LimeTextExplainer
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from datasets import load_dataset

# Step 1: Load the fine-tuned model and tokenizer
model_name = "Davlan/xlm-roberta-base-ner-hrl"  # Replace with your model's path or Hugging Face model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create an NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Step 2: Load or define a dataset
# Replace this with your dataset loading step (e.g., loading from .conll files)
sample_texts = [
    "አዲስ አበባ ውስጥ እቃዎችን በዝቅተኛ ዋጋ ማግኘት ይቻላል።",
    "በአዲስ አበባ የእንደቁልፍ ቤቶችን መታየት ይቻላል።",
    "ኢትዮጵያ ውስጥ እቃዎችን ለእንቅስቃሴ እንደተዘጋጅቷቸው ታውቋል።"
]

# Step 3: Interpret with SHAP
# Wrap the model for SHAP compatibility
def ner_predict(inputs):
    tokenized_inputs = tokenizer(inputs, truncation=True, return_tensors="pt", padding=True)
    outputs = model(**tokenized_inputs)
    logits = outputs.logits.detach().numpy()
    return logits

# Use SHAP explainer
explainer = shap.Explainer(ner_predict, sample_texts)
shap_values = explainer(sample_texts)

# Visualize SHAP explanations for each text
for i, text in enumerate(sample_texts):
    print(f"SHAP Explanation for Text {i + 1}:")
    shap.plots.text(shap_values[i])

# Step 4: Interpret with LIME
# Define a LIME-compatible prediction function
def lime_predict(inputs):
    results = []
    for input_text in inputs:
        result = ner_pipeline(input_text)
        label_scores = {}
        for entity in result:
            label_scores[entity["entity_group"]] = entity["score"]
        results.append(label_scores)
    return results

# Use LIME explainer
explainer = LimeTextExplainer(class_names=model.config.id2label.values())

# Explain a specific example with LIME
text_to_explain = sample_texts[0]
explanation = explainer.explain_instance(text_to_explain, lime_predict, num_features=6)
explanation.show_in_notebook()

# Step 5: Analyze difficult cases
# Example of analyzing predictions
for i, text in enumerate(sample_texts):
    print(f"Text {i + 1}: {text}")
    predictions = ner_pipeline(text)
    print("Predictions:")
    for pred in predictions:
        print(pred)
    print("-" * 50)

# Step 6: Generate interpretability report
# Use SHAP and LIME insights to create a table (can be exported to a CSV or visualized)
import pandas as pd

report_data = []
for text in sample_texts:
    predictions = ner_pipeline(text)
    for pred in predictions:
        report_data.append({
            "Text": text,
            "Entity": pred["entity_group"],
            "Word": pred["word"],
            "Confidence": pred["score"],
            "Start": pred["start"],
            "End": pred["end"]
        })

# Convert to a DataFrame for better visualization
report_df = pd.DataFrame(report_data)
print("Interpretability Report:")
print(report_df)
